## WARNING:
### All paths in this notebook are set up for use in Colab. If a local system is used, they must be adapted to the environment.

In [ ]:
import zipfile
from pathlib import Path

from google.colab import files

## Check for GPU Usage

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != "/device:GPU:0":
    raise SystemError(
        "\n\nThis error most likely means that this notebook is not "
        "configured to use a GPU.  Change this in Notebook Settings via the "
        "command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n"
    )
print("Found GPU at: {}".format(device_name))

## Install necessary modules
Only neccessary if you working in colab. If you use your local system, please use the requirements.txt

In [ ]:
! pip -q install pytorch-lightning --upgrade
! pip -q install neptune-client
! pip -q install kornia
! pip -q install split-folders

## Clone files from Github

In [ ]:
! git clone https://github.com/RobinMaas95/GTSRB_Visualization.git

## Download Datasets

In [ ]:
! python GTSRB_Visualization/code/get_datasets.py --crop True

## Initialize Datasets

In [ ]:
! python GTSRB_Visualization/code/initialize_dataset.py \
    --source_path "/content/data/cropped_training" \
    --target_path "/content/data/cropped_training_init"

## Build Model

In [ ]:
! rm -rf checkpoints/

In [ ]:
! python /content/GTSRB_Visualization/code/build_model.py \
  --train_dataset "/content/data/cropped_training_init" \
  --test_dataset "/content/data/cropped_test"

## Set mean/std variables
Set mean/std variables. 
You find them in the logs above close the top,
just before the training begins.
E.g.:

```
/content/GTSRB_Visualization/code/build_model.py - INFO - Mean: [0.32352597020361307, 0.2917746800988572, 0.31248061517888454]
/content/GTSRB_Visualization/code/build_model.py - INFO - Std: [0.2770588879067902, 0.2602109151608635, 0.2735251700292777]
```

Replace the values of the _list variables with the lists at the end of the log lines:

 

In [ ]:
mean_list = [0.3232291265307408, 0.2916548292210219, 0.31239060828717274]
std_list = [0.2765753268041087, 0.26007308085671965, 0.27338755423173944]
mean = f"{mean_list[0]} {mean_list[1]} {mean_list[2]}"
std = f"{std_list[0]} {std_list[1]} {std_list[2]}"

## Run visualizations
Saliency Map and Activation Maximization use the code from the following repo:

https://github.com/hans66hsu/nn_interpretability

Unfortunately, the package versions required to do this cause CUDA to stop working. Therefore GradCam and GradCam++ are first run with a version of visualization.py that does not import nn_interpretability.

In [ ]:
# Autoselect best checkpoint
from pathlib import Path

checkpoint_results_path = "/content/checkpoints/lightning_logs/version_0/checkpoints/"
p = Path(checkpoint_results_path).glob("**/*")
files = [x for x in p if x.is_file()]
for file in files:
    if file.name != "last.ckpt":
        best_checkpoint = str(file)

In [ ]:
Path(r"/content/visualizations").mkdir(exist_ok=True)
! python /content/GTSRB_Visualization/code/visualization.py \
  --dest "/content/visualizations/" --filetype "ppm" \
  --model $best_checkpoint \
  --src "/content/data/cropped_test" --vis "GradCam" \
  --mean $mean \
  --std $std

In [ ]:
Path(r"/content/visualizations").mkdir(exist_ok=True)
! python /content/GTSRB_Visualization/code/visualization.py \
  --dest "/content/visualizations/" --filetype "ppm" \
  --model $best_checkpoint \
  --src "/content/data/cropped_test" --vis "GradCam++" \
  --mean $mean \
  --std $std

In [ ]:
# Install nn_interpretability, make sure you push the restart button!
! pip install -e "/content/GTSRB_Visualization/code/nn_interpretability"

In [ ]:
Path(r"/content/visualizations").mkdir(exist_ok=True)
! python /content/GTSRB_Visualization/code/visualization_with_nn.py \
  --dest "/content/visualizations/" --filetype "ppm" \
  --model $best_checkpoint \
  --src "/content/data/cropped_test" --vis "Saliency" \
  --mean $mean \
  --std $std

In [ ]:
# Activation maximization does not really seem to deliver reproducible results. 
# Sometimes the accuracy for individual classes drops dramatically and running 
# it again fixes the problem. Therefore it is possible that the values generated
# here differ more from the values of the thesis than with the other methods.
Path(r"/content/visualizations").mkdir(exist_ok=True)
! python /content/GTSRB_Visualization/code/visualization_with_nn.py \
  --dest "/content/visualizations/" --filetype "ppm" \
  --model $best_checkpoint \
  --src "/content/data/cropped_test" --vis "Activation Maximation" \
  --mean $mean \
  --std $std

## Mask images

In [ ]:
# For our masking, we only need the following three folders.
# We move them to an extra folder, for easier looping
import shutil

target = Path("/content/to_be_masked/")
target.mkdir(exist_ok=True, parents=True)
shutil.move("/content/visualizations/heatmap_grad_cam", target)
shutil.move("/content/visualizations/heatmap_grad_cam_pp", target)
shutil.move("/content/visualizations/heatmap_saliency", target)

In [ ]:
! python /content/GTSRB_Visualization/code/mask_images.py --heatmaps "/content/to_be_masked/" \
--json_target /content/masking_jsons --json_file _heatmap_masked.csv \
--org_images "/content/data/cropped_test" \
--target /content/masking_results/  \

## Download files
Download the created datasets and results. Not necessary if you use a local system.

### Datasets

In [ ]:
%cd /content

In [ ]:
! zip -r -q cropped_training_init.zip /content/data/cropped_training_init
! zip -r -q cropped_test.zip /content/data/cropped_test

! mkdir /content/datasets
! mv cropped_training_init.zip /content/datasets
! mv cropped_test.zip /content/datasets

! zip -r -q datasets.zip /content/datasets/

Download File or copy it to personal Google Drive
(Make sure your Google Drive is connected)

In [ ]:
# Example for download
# files.download(datasets.zip) 

# Example for Drive
! cp datasets.zip /content/drive/MyDrive/

### Results

In [ ]:
! zip -r -q checkpoints.zip  /content/checkpoints
! zip -r -q masking_jsons.zip  /content/masking_jsons
! zip -r -q to_be_masked.zip  /content/to_be_masked
! zip -r -q visualizations.zip  /content/visualizations/
! zip -r -q masking_results.zip /content/masking_results

! mkdir /content/results
! mv checkpoints.zip  /content/results
! mv masking_jsons.zip  /content/results
! mv to_be_masked.zip  /content/results
! mv visualizations.zip /content/results
! mv masking_results.zip /content/results

! zip -r -q results_nesterov.zip  /content/results